In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

import os

# 1. Create a function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(kaggle_file, ratings_file, wiki_file):
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file)
    ratings = pd.read_csv(ratings_file)

    # 3. Open the read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as wikifile:
        wiki_movies_raw = json.load(wikifile)
        
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    
    # 5. Return the three DataFrames
    return wiki_movies_raw, wiki_movies_df, kaggle_metadata, ratings

# 6 Create the path to your file directory and variables for the three files. 
file_dir = os.path.join('..','..','8_repo','bootcamp8_Movies-ETL','Resources')
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

# 7. Set the three variables in Step 6 equal to the function created in Step 1.
wiki_movies_raw, wiki_file, kaggle_file, ratings_file = extract_transform_load(kaggle_file=kaggle_file, ratings_file=ratings_file, wiki_file=wiki_file)

C:\Anaconda3\envs\databootcamp\lib\site-packages\IPython\core\interactiveshell.py:3263: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
wiki_movies_raw[0]

{'url': 'https://en.wikipedia.org/wiki/The_Adventures_of_Ford_Fairlane',
 'year': 1990,
 'imdb_link': 'https://www.imdb.com/title/tt0098987/',
 'title': 'The Adventures of Ford Fairlane',
 'Directed by': 'Renny Harlin',
 'Produced by': ['Steve Perry', 'Joel Silver'],
 'Screenplay by': ['David Arnott', 'James Cappe', 'Daniel Waters'],
 'Story by': ['David Arnott', 'James Cappe'],
 'Based on': ['Characters', 'by Rex Weiner'],
 'Starring': ['Andrew Dice Clay',
  'Wayne Newton',
  'Priscilla Presley',
  'Lauren Holly',
  'Morris Day',
  'Robert Englund',
  "Ed O'Neill"],
 'Narrated by': 'Andrew "Dice" Clay',
 'Music by': ['Cliff Eidelman', 'Yello'],
 'Cinematography': 'Oliver Wood',
 'Edited by': 'Michael Tronick',
 'Productioncompany ': 'Silver Pictures',
 'Distributed by': '20th Century Fox',
 'Release date': ['July 11, 1990', '(', '1990-07-11', ')'],
 'Running time': '102 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$20 million',
 'Box office': '$21.4 milli

In [27]:
for i in range(len(wiki_movies_raw)):
    try:
        print(wiki_movies_raw[i]['No. of episodes'], end='')
        print(type(wiki_movies_raw[i]['No. of episodes']), end='')
        print(f' - movie # {i}')
    except:
        continue
print(i)

14<class 'str'> - movie # 71
1<class 'str'> - movie # 497
4<class 'str'> - movie # 1148
['26', '(', 'list of episodes', ')']<class 'list'> - movie # 2082
1<class 'str'> - movie # 2124
10<class 'str'> - movie # 2668
7310


In [35]:
'No. of episodes' not in wiki_movies_raw[70]

True

In [46]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)
# wiki_movies_df
[(column, wiki_movies_df[column].isnull().sum()) for column in wiki_movies_df.columns]

[('url', 158),
 ('year', 158),
 ('imdb_link', 211),
 ('title', 177),
 ('Directed by', 208),
 ('Produced by', 571),
 ('Screenplay by', 4987),
 ('Story by', 6306),
 ('Based on', 5112),
 ('Starring', 391),
 ('Narrated by', 7026),
 ('Music by', 808),
 ('Cinematography', 911),
 ('Edited by', 913),
 ('Productioncompany ', 2771),
 ('Distributed by', 712),
 ('Release date', 397),
 ('Running time', 342),
 ('Country', 585),
 ('Language', 438),
 ('Budget', 2536),
 ('Box office', 1781),
 ('Written by', 2714),
 ('Genre', 7182),
 ('Theme music composer', 7241),
 ('Country of origin', 7153),
 ('Original language(s)', 7147),
 ('Producer(s)', 7150),
 ('Editor(s)', 7160),
 ('Production company(s)', 7175),
 ('Original network', 7180),
 ('Original release', 7146),
 ('Productioncompanies ', 6582),
 ('Executive producer(s)', 7210),
 ('Production location(s)', 7261),
 ('Distributor', 7171),
 ('Picture format', 7242),
 ('Audio format', 7247),
 ('Voices of', 7308),
 ('Followed by', 7297),
 ('Composer(s)', 7300

In [43]:
print(f'{wiki_movies_df["No. of episodes"].isnull().sum()} vs {len(wiki_movies_df)}')


7305 vs 7311


In [48]:
box_office = wiki_movies_df['Box office'].dropna()
box_office.sample(20)

819                             $52.2 million
5127                              $69 million
4257             [$495,303, (worldwide), [1]]
4359                   [$158.9, million, [2]]
5422                           $139.8 million
3350                            $47.5 million
3030                               $4,695,781
3105                            $65.8 million
5444                           $211.8 million
818                             $76.2 million
3828                            $97.9 million
4558                           $340.5 million
740                            $159.1 million
4007                                 $519,802
1788                             $173 million
222                                   $43,982
2472                            $11.7 million
261     [$14.6 million, (North America), [3]]
7020                              $13 million
4015                            $59.9 million
Name: Box office, dtype: object

In [53]:
wiki_movies_df['imdb_link']

0       https://www.imdb.com/title/tt0098987/
1       https://www.imdb.com/title/tt0098994/
2       https://www.imdb.com/title/tt0099005/
3       https://www.imdb.com/title/tt0099012/
4       https://www.imdb.com/title/tt0099018/
                        ...                  
7306    https://www.imdb.com/title/tt1255919/
7307    https://www.imdb.com/title/tt6266538/
7308    https://www.imdb.com/title/tt4669788/
7309    https://www.imdb.com/title/tt7137380/
7310    https://www.imdb.com/title/tt9495224/
Name: imdb_link, Length: 7311, dtype: object

In [70]:
movielink=wiki_movies_df.loc[0,'imdb_link']
to_look_for = r'tt\d{7}'
re.search(to_look_for, movielink).group(0)

'tt0098987'

In [72]:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')

In [73]:
#     try:
wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        
#     except:
#         print('error')

In [74]:
wiki_movies_df['imdb_id']

0       tt0098987
1       tt0098994
2       tt0099005
3       tt0099012
4       tt0099018
          ...    
7306    tt1255919
7307    tt6266538
7308    tt4669788
7309    tt7137380
7310    tt9495224
Name: imdb_id, Length: 7056, dtype: object

In [89]:
box_office = wiki_movies_df['Box office'].dropna()

# 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)


# 10. Write a regular expression to match the six elements of "form_one" of the box office data.
box_office_form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
box_office_form_two = r'\$\s*\d{1,3}(?:,\d{3})+'
# box_office_form_two = r'\$\s*\d{1,3}(?:,\d{3})*(?:\.000)?'

matches_form_one = box_office.str.contains(box_office_form_one, flags=re.I, na=False)
matches_form_two = box_office.str.contains(box_office_form_two, flags=re.I, na=False)

box_office[~matches_form_one & ~matches_form_two]

82                         $335.000
113              $4.35-4.37 million
618                      $5000 (US)
1113                     35,254,617
1541                     £3 million
1679                       $520.000
1952                   ¥1.1 billion
2132                            N/A
2193                           $309
2458                       $282.175
2784    926,423 admissions (France)
2946                       $414.000
3050                       $621.000
3222      $32 [2] –33.1 million [1]
3774                            TBA
4006             $38.9–40.3 million
4026       CN¥3.650 million (China)
4270                     £7,385,434
4279                       $161.000
4426             $20.7–23.9 million
4476                         $20-30
4746        $45.2k (only in Turkey)
5645                          £2.56
5983                       413 733$
6222                        Unknown
6359             $17.5–18.4 million
6449             $41.8–41.9 million
6587                        

In [91]:
print(len(box_office[~matches_form_one & ~matches_form_two]),end=' ')
print(len(box_office))

31 5484


In [92]:
    def parse_dollars(s):
        if type(s) != str:
            return np.nan
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.I):
            s = re.sub('\$|\s|[a-zA-Z]', '', s)
            value = float(s) * 1e6
            return value
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.I):
            s = re.sub('\$|\s|[a-zA-Z]', '', s)
            value = float(s) * 1e9
            return value
        elif re.match(r'\$\s*\d{1,3}(?:,\d{3})+', s, flags=re.I):
            s = re.sub('\$|\s|,', '', s)
            value = float(s) 
            return value
        else:
            return np.nan

In [119]:
budget_form_one = r'\$\s*\d+\.?\d*\s*[mb]il?li?on'
budget_form_two = r'\$\s*\d{1,3}(?:,\d{3})*(?:\.0{1,2})?'
budget_form_three = r'\$\d{1,3}(?:\.\d)?[-—–]\d{1,3}(?:\.\d)?\smillion'

budget = wiki_movies_df['Budget'].dropna()
budget = budget.map(lambda x: ' '.join(x) if type(x)==list else x)

matches_form_one = budget.str.contains(budget_form_one, flags=re.I, na=False)
matches_form_two = budget.str.contains(budget_form_two, flags=re.I, na=False)
matches_form_three = budget.str.contains(budget_form_three, flags=re.I, na=False)

budget[~matches_form_one & ~matches_form_two & ~matches_form_three]

139                         Unknown
210     60 million Norwegian Kroner
490                         Unknown
1276                        Unknown
1331                            HBO
1429                     £6,000,000
1455                     13 million
1541                   £2.8 million
1817                   CAD2,000,000
2002     PHP 85 million (estimated)
2038                    102,888,900
2043                   3,500,000 DM
2064                     ₤2,300,874
2563                     ₤6,350,000
3278                   € 40 million
3947                   £4.2 million
4053                            N/A
4108                    760,000 USD
4648                       19 crore
4827                    £17 million
5621                            N/A
5645                     £4 million
5870                    €14 million
6603               £ 12 million [3]
6813                     £3 million
7047                  £12.9 million
7070                      3.5 crore
7124                        

In [130]:
budget_to_test = budget[matches_form_three]
budget_to_test

113                     $5–7 million
169                   $15-20 million
201     $6.5–10 million [1] [Note 1]
242                   $50–65 million
308                   $12–15 million
                    ...             
7291                  $6–7.7 million
7297                $100–150 million
7301                $160–200 million
7302                $102–135 million
7303                  $39–50 million
Name: Budget, Length: 235, dtype: object

In [144]:
re.search(r'\$\d{1,3}(?:\.\d)?[-—–](\d{1,3}(?:\.\d)?)(?=\smillion)', budget_to_test[7302], flags=re.I).group(1)

'135'

In [141]:
type(re.search(r'\$\d{1,3}(?:\.\d)?[-—–](\d{1,3}(?:\.\d)?)(?=\smillion)', budget_to_test[201], flags=re.I).group(1))

str

In [145]:
sorted([(column, wiki_movies_df[column].isnull().sum()) for column in wiki_movies_df.columns])

[('Actor control', 7056),
 ('Adaptation by', 7054),
 ('Alias', 7055),
 ('Alma mater', 7055),
 ('Also known as', 7054),
 ('Animation by', 7054),
 ('Arabic', 7054),
 ('Area', 7055),
 ('Area served', 7056),
 ('Artist(s)', 7056),
 ('Attraction type', 7055),
 ('Audio format', 6992),
 ('Author', 7052),
 ('Based on', 4872),
 ('Biographical data', 7056),
 ('Bopomofo', 7056),
 ('Born', 7056),
 ('Box office', 1572),
 ('Budget', 2318),
 ('Camera setup', 7055),
 ('Cantonese', 7055),
 ('Characters', 7055),
 ('Children', 7056),
 ('Chinese', 7055),
 ('Cinematography', 711),
 ('Closing date', 7055),
 ('Color process', 7055),
 ('Comics', 7056),
 ('Composer(s)', 7046),
 ('Coordinates', 7055),
 ('Country', 403),
 ('Country of origin', 6898),
 ('Cover artist', 7055),
 ('Created by', 7042),
 ('Date premiered', 7055),
 ('Designer(s)', 7056),
 ('Developed by', 7055),
 ('Developer(s)', 7056),
 ('Dewey Decimal', 7056),
 ('Died', 7056),
 ('Directed by', 22),
 ('Director', 7054),
 ('Distributed by', 518),
 ('Dis

In [147]:
release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x)==list else x)
wiki_movies_df['Release date']

0                       [July 11, 1990, (, 1990-07-11, )]
1       [May 17, 1990, (, 1990-05-17, ), (Cannes Film ...
2                     [August 10, 1990, (, 1990-08-10, )]
3                   [December 25, 1990, (, 1990-12-25, )]
4                                       December 19, 1990
                              ...                        
7306    [December 25, 2018, (, 2018-12-25, ), (United ...
7307    [December 11, 2018, (, 2018-12-11, ), (, Samue...
7308    [November 8, 2018, (, 2018-11-08, ), (, AFI Fe...
7309    [August 31, 2018, (, 2018-08-31, ), (, Telluri...
7310                 [28 December 2018, (, 2018-12-28, )]
Name: Release date, Length: 7056, dtype: object

In [148]:
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
date_form_two = r'\d{4}.[01]\d.[0123]\d' # matches yyyy-mm-dd and yyyy/mm/dd
date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
date_form_four = r'\d{4}'
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
wiki_movies_df['release_date'].sample(20)

1499   2000-09-12
2887   2000-09-08
89     1989-10-17
5664   2012-08-03
6179   2013-04-06
3635   2004-09-10
1327   1995-09-06
5454   2011-04-28
6632   2015-07-30
4254   2006-01-01
735    1993-12-03
4497   2007-09-21
6405   2014-06-05
6705   2016-03-15
606    1992-11-09
2525   1999-12-10
2741   2000-03-03
237    1990-11-21
815    1993-11-12
3502   2003-12-19
Name: release_date, dtype: datetime64[ns]

In [162]:
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2]==0 else row[2], axis=1) # axis=1 -> columns
wiki_movies_df['running_time'].sample(20)

2512    101.0
5008     94.0
4883    115.0
330      89.0
4067     78.0
4318    132.0
4044    107.0
5990     94.0
332     100.0
7076    127.0
4087    105.0
3874    122.0
4838    122.0
6521     81.0
1109    133.0
5567     79.0
5336    115.0
3847    114.0
5366    110.0
1213    122.0
Name: running_time, dtype: float64

In [94]:
wiki_movies_df['box_office'] = box_office.str.extract(f'({box_office_form_one}|{box_office_form_two})')[0].apply(parse_dollars)
wiki_movies_df['box_office']

0       21400000.0
1        2700000.0
2       57718089.0
3        7331647.0
4        6939946.0
           ...    
7306    41900000.0
7307    76100000.0
7308    38400000.0
7309     5500000.0
7310           NaN
Name: box_office, Length: 7056, dtype: float64

In [96]:

wiki_movies_df['budget'] = box_office.str.extract(f'({box_office_form_one}|{box_office_form_two})')[0].apply(parse_dollars)
# wiki_movies_df['budget']

0       21400000.0
1        2700000.0
2       57718089.0
3        7331647.0
4        6939946.0
           ...    
7306    41900000.0
7307    76100000.0
7308    38400000.0
7309     5500000.0
7310           NaN
Name: budget, Length: 7056, dtype: float64

In [77]:
box_office

0          $21.4 million
1           $2.7 million
2            $57,718,089
3             $7,331,647
4       $6,939,946 (USA)
              ...       
7305       $19.4 million
7306       $41.9 million
7307       $76.1 million
7308       $38.4 million
7309        $5.5 million
Name: Box office, Length: 5484, dtype: object

In [2]:
# 8. Set the DataFrames from the return statement equal to the file names in Step 6. 
wiki_movies_df = wiki_file
kaggle_metadata = kaggle_file
ratings = ratings_file

In [3]:
# 9. Check the wiki_movies_df DataFrame.
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Predecessor,Founders,Area served,Products,Services,Russian,Hebrew,Revenue,Operating income,Polish
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# 10. Check the kaggle_metadata DataFrame.
kaggle_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
# 11. Check the ratings DataFrame.
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
